# [ Pré-processamento ] Heurísticas computacionais para extração deconhecimento em problemas de maratona deprogramação - Parte 2 (PLN)

Após realizar o tratamento básico dos dados, é necessário realizar alguns passos básicos de Processamento de Linguagem Natural (PLN) a fim de poder fornecer os problemas como entrada para algum classificador.

### Importando as bibliotecas necessárias

Antes de começar, é necessário importar algumas bibliotecas para tratar os dados.

In [1]:
import os
import json
import nltk
import numpy as np
import pandas as pd

from nltk.corpus import stopwords

### Declarando constantes de trabalho

Criação de "variáveis de ambiente" para auxiliar a obtenção de dados.

In [2]:
# Langs? {PT, EN, ES}
LANG_SLUG = "EN"
PREPROCESSED_DATA_DIR = "../Datasets/"

# Preprocessed files {%LANG_00_raw_data, %LANG_01_primary_data, %LANG_02_secondary_data, %LANG_03_terciary_data}.csv
PREPROCESSED_FILE = "%s%s%s" % (PREPROCESSED_DATA_DIR, LANG_SLUG, "_03_terciary_data.csv")

### Tokenização

Aqui os textos de descrição, entrada e saída serão tokenizados (extraídos as palavras)

In [3]:
# Lê o arquivo CSV e trata os objetos vazios
rawDataDF = pd.read_csv(PREPROCESSED_FILE, sep=";", index_col = 0)
rawDataDF.fillna("", inplace = True)

# Tokeniza os textos do problema
tokenized = pd.DataFrame()
tokenized["problem"] = rawDataDF.apply(lambda x : nltk.word_tokenize(x["description"]), axis = 1)
tokenized["in"] = rawDataDF.apply(lambda x : nltk.word_tokenize(x["input"]), axis = 1)
tokenized["out"] = rawDataDF.apply(lambda x : nltk.word_tokenize(x["output"]), axis = 1)

# Converte todas as letras para "minúsculas"
tokenized["problem"] = tokenized.apply(lambda x: [w.lower() for w in x["problem"]], axis = 1)
tokenized["in"] = tokenized.apply(lambda x: [w.lower() for w in x["in"]], axis = 1)
tokenized["out"] = tokenized.apply(lambda x: [w.lower() for w in x["out"]], axis = 1)

# Imprime parte do DF
tokenized.head()

,problem,in,out
1001,"[read, 2, variables, ,, named, a, and, b, and,...","[the, input, file, will, contain, 2, integer, ...","[print, the, letter, x, (, uppercase, ), with,..."
1002,"[the, formula, to, calculate, the, area, of, a...","[the, input, contains, a, value, of, floating,...","[present, the, message, ``, a=, '', followed, ..."
1003,"[read, two, integer, values, ,, in, this, case...","[the, input, file, contains, 2, integer, numbe...","[print, the, variable, soma, with, all, the, c..."
1004,"[read, two, integer, values, ., after, this, ,...","[the, input, file, contains, 2, integer, numbe...","[print, prod, according, to, the, following, e..."
1005,"[read, two, floating, points, ', values, of, d...","[the, input, file, contains, 2, floating, poin...","[print, media, (, average, in, portuguese, ), ..."


### Remoção de stopwords

Remoção de stopwords e limpeza de caracteres (remoção de palavras não alfanuméricas) com o auxílio do NLTK

In [4]:
stoplist = stopwords.words('english')

tokenized["problem"] = tokenized.apply(lambda x: [w for w in x["problem"] if (w not in stoplist and w.isalpha())], axis = 1)
tokenized["in"] = tokenized.apply(lambda x: [w for w in x["in"] if (w not in stoplist and w.isalpha())], axis = 1)
tokenized["out"] = tokenized.apply(lambda x: [w for w in x["out"] if (w not in stoplist and w.isalpha())], axis = 1)

# Imprime parte do DF
tokenized.head()

,problem,in,out
1001,"[read, variables, named, b, make, sum, two, va...","[input, file, contain, integer, numbers]","[print, letter, x, uppercase, blank, space, eq..."
1002,"[formula, calculate, area, circumference, defi...","[input, contains, value, floating, point, doub...","[present, message, followed, value, variable, ..."
1003,"[read, two, integer, values, case, variables, ...","[input, file, contains, integer, numbers]","[print, variable, soma, capital, letters, blan..."
1004,"[read, two, integer, values, calculate, produc...","[input, file, contains, integer, numbers]","[print, prod, according, following, example, b..."
1005,"[read, two, floating, points, values, double, ...","[input, file, contains, floating, points, valu...","[print, media, average, portuguese, according,..."


### Aplicação de tag - Part Of Speech (POS)

Aplicação de POS tags utilizando a NLTK

In [5]:
# Aplica o POS_TAG
tagged = pd.DataFrame()
tagged["problem"] = tokenized.apply(lambda x: nltk.pos_tag(x["problem"]), axis = 1)
tagged["in"] = tokenized.apply(lambda x: nltk.pos_tag(x["in"]), axis = 1)
tagged["out"] = tokenized.apply(lambda x: nltk.pos_tag(x["out"]), axis = 1)

# Imprime parte do DF
tagged.head()

,problem,in,out
1001,"[(read, JJ), (variables, NNS), (named, VBN), (...","[(input, NN), (file, NN), (contain, NN), (inte...","[(print, NN), (letter, NN), (x, NNP), (upperca..."
1002,"[(formula, NN), (calculate, NN), (area, NN), (...","[(input, NN), (contains, VBZ), (value, NN), (f...","[(present, JJ), (message, NN), (followed, VBD)..."
1003,"[(read, VB), (two, CD), (integer, JJR), (value...","[(input, NN), (file, NN), (contains, VBZ), (in...","[(print, NN), (variable, JJ), (soma, NN), (cap..."
1004,"[(read, VB), (two, CD), (integer, JJR), (value...","[(input, NN), (file, NN), (contains, VBZ), (in...","[(print, NN), (prod, NN), (according, VBG), (f..."
1005,"[(read, VB), (two, CD), (floating, VBG), (poin...","[(input, NN), (file, NN), (contains, VBZ), (fl...","[(print, NN), (media, NNS), (average, JJ), (po..."


### Reconhecimento de entidades nomeadas

Aplicação de reconhecimento de entidades nomeadas utilizando a NLTK

In [7]:
# Aplica o reconhecimento de entidades nomeadas
named_entities = pd.DataFrame()
named_entities["problem"] = tagged.apply(lambda x: nltk.ne_chunk(x["problem"], binary = False), axis = 1)
named_entities["in"] = tagged.apply(lambda x: nltk.ne_chunk(x["in"], binary = False), axis = 1)
named_entities["out"] = tagged.apply(lambda x: nltk.ne_chunk(x["out"], binary = False), axis = 1)

# Imprime parte do DF
named_entities.head()

,problem,in,out
1001,"[(read, JJ), (variables, NNS), (named, VBN), (...","[(input, NN), (file, NN), (contain, NN), (inte...","[(print, NN), (letter, NN), (x, NNP), (upperca..."
1002,"[(formula, NN), (calculate, NN), (area, NN), (...","[(input, NN), (contains, VBZ), (value, NN), (f...","[(present, JJ), (message, NN), (followed, VBD)..."
1003,"[(read, VB), (two, CD), (integer, JJR), (value...","[(input, NN), (file, NN), (contains, VBZ), (in...","[(print, NN), (variable, JJ), (soma, NN), (cap..."
1004,"[(read, VB), (two, CD), (integer, JJR), (value...","[(input, NN), (file, NN), (contains, VBZ), (in...","[(print, NN), (prod, NN), (according, VBG), (f..."
1005,"[(read, VB), (two, CD), (floating, VBG), (poin...","[(input, NN), (file, NN), (contains, VBZ), (fl...","[(print, NN), (media, NNS), (average, JJ), (po..."
